# <u>Sujet 1 : Evaluation de deux plateformes open source d'analyse linguistique</u>

## <u> I - Evaluation de l'analyse morpho-syntaxique </u>

##### 1) Utiliser le corpus annoté « pos_reference.txt.lima » pour extraire les phrases ayant servi pour produire ce corpus annoté et sauvegarder le résultat dans le fichier « pos_test.txt  ».

In [24]:
def getText(fileIn, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')

    for line in fileI:

        tuples = line.split()
        if tuples:
            # On souhaite récupérer l'entièreté de la ligne à l'exception du dernier mot
            # qui est l'annotation
            for i in range(len(tuples)-1):
                fileO.write(tuples[i]+" ")
        else:
            # Si on arrive à une ligne vide on saute une ligne c'est un changement de phrase 
            fileO.write("\n")

    fileI.close()
    fileO.close()


In [25]:
fileIn = "pos_reference.txt.lima"
fileOut = "pos_test.txt"

getText(fileIn,fileOut)


##### 2) Convertir  les  tags  du  corpus  annoté  « pos_reference.txt.lima »  en  tags  universels  et sauvegarder le résultat dans le fichier « pos_reference.txt.univ ». Il faut utiliser les deux tables de correspondance « POSTags_LIMA_PTB_Linux.txt » et « POSTags_PTB_Universal_Linux.txt » pour réaliser cette transformation (LIMA => PTB => Universal). 
 

In [26]:
def setTagUniv(fileIn, fileRef1,fileRef2, fileOut):
    fileI = open(fileIn,'r')
    fileR1 = open(fileRef1,'r')
    fileR2 = open(fileRef2,'r')
    fileO = open(fileOut,'w')

    my_dico1 = {}
    my_dico2 = {}

    #on crée deux dictionnaires à partir des tags de reference
    for line in fileR1:
        tuples = line.split()
        my_dico1[tuples[0]] = tuples[1]
    fileR1.close()

    for line in fileR2:
        tuples = line.split()
        my_dico2[tuples[0]] = tuples[1]
    fileR2.close()

    # modification des tags vers les tags univ
    for l in fileI:
        tup = l.split()
        if tup:
            # on écrit le groupe de mots initial
            for i in range(len(tup)-1):
                    fileO.write(tup[i]+" ")
            temp = my_dico1[tup[len(tup)-1]]

            fileO.write('\t'+ my_dico2[temp])
            fileO.write('\n')
        else:
            fileO.write("\n")

    fileI.close()
    fileO.close()

In [27]:
fileIn = "pos_reference.txt.lima"
fileRef1 = "POSTags_LIMA_PTB_Linux.txt"
fileRef2 = "POSTags_PTB_Universal_Linux.txt"
fileOut = "pos_reference.txt.univ"


setTagUniv(fileIn,fileRef1,fileRef2,fileOut)


##### 3) Lancer les deux POS taggers sur le fichier « pos_test.txt ». Les résultats doivent avoir le format du  corpus  annoté  « pos_reference.txt.lima »  (2  colonnes  séparées  par  une  tabulation)  et doivent être sauvegardés respectivement dans les fichiers suivants : 
-  pos_test.txt.pos.stanford 
-  pos_test.txt.pos.nltk 

In [28]:
import nltk
from nltk.tokenize import word_tokenize

def nltkTokenize(fileIn,fileOut):
    
    # Ouvrir le fichier en lecture seule
    file = open(fileIn, "r")
    fileO = open(fileOut, "w")
    for line in file:
        
        text = word_tokenize(line)
    
        tokens_tag = nltk.pos_tag(text)
        
        for pair in tokens_tag :
            fileO.write('\t'.join(pair))  
            fileO.write('\n')
    fileO.close()
    file.close()   

Ligne de commande à taper dans le terminal linux depuis le dossier stanford-postagger

./stanford-postagger.sh models/english-left3words-distsim.tagger ../pos_test.txt > ../pos_test.txt.pos.stanford

In [29]:
fileIn = "pos_test.txt"
fileOutNltk = "pos_test.txt.pos.nltk"
fileOutStan = "pos_test.txt.pos.stanford"


nltkTokenize(fileIn,fileOutNltk)


##### 4) Convertir les résultats des deux POS taggers en utilisant les étiquettes universelles (Annexe 1). Il faut utiliser la table de correspondance « POSTags_PTB_Universal_Linux.txt » pour réaliser cette  transformation  (PTB => Universal).  Les  résultats  de  cette  conversion  doivent  être sauvegardés respectivement dans les fichiers suivants : 
-  pos_test.txt.pos.stanford.univ 
-  pos_test.txt.pos.nltk.univ

In [30]:
# Création du dictionnaire PTB vers Universal
my_dico = {}

#on crée un dictionnaire à partir des tags de reference
fileR = open(fileRef2,'r')
for line in fileR:
    tuples = line.split()
    my_dico[tuples[0]] = tuples[1]
fileR.close()




In [31]:
def setTagPTBUnivNltk(fileIn, dico, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')

    
    # modification des tags vers les tags univ
    for l in fileI:
        tup = l.split()
        if tup:
            # on écrit le groupe de mots initial
            fileO.write(tup[0]+'\t'+ dico[tup[1]])
           
            fileO.write('\n')
        else:
            fileO.write("\n")

    fileI.close()
    fileO.close()

In [32]:
setTagPTBUnivNltk(fileOutNltk, my_dico, "pos_test.txt.pos.nltk.univ")

##### Ajout d'étiquettes dans le fichier PTB vers Univ car nous avons constater un manque pour les étiquettes suivantes :
-  $ => CUR
-  `` => .
-  '' => .
-  -LRB- => .
-  -RRB- => .

#### Modification d'un passage dans pos_reference.txt.lima qui fait planter le pos tagger de stanford

 - ligne 3293 => 3 1/2 NUM    ->  3 NUM

In [35]:
def setTagPTBUnivStan(fileIn, dico, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')
    checkTag = 0
    w = ""
    for l in fileI:
        tup = l.split()
        for word in tup:
            #print(word)
            for letter in word:
                if(letter == "_"):
                    fileO.write("\t")
                    checkTag = 1
                if(checkTag == 1 and letter != "_"):
                    w = w+letter
                if(letter != "_" and checkTag == 0):
                    fileO.write(letter)
            fileO.write(" "+dico[w]+" ")
            fileO.write('\n')
            checkTag = 0
            w = ""
    fileI.close()
    fileO.close()   
    

In [36]:
setTagPTBUnivStan(fileOutStan,my_dico,"pos_test.txt.pos.stanford.univ")

##### 5) Lancer l’évaluation des deux POS taggers. Pour réaliser cette évaluation, il faut supprimer la ligne vide (séparant les phrases) dans le de fichier de référence « pos_reference.txt.univ » : 
-  python evaluate.py pos_test.txt.pos.stanford.univ pos_reference.txt.univ 
-  python evaluate.py pos_test.txt.pos.nltk.univ pos_reference.txt.univ 

In [3]:
def suppressEmptyLine(fileIn, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')

    for line in fileI:
        tup = line.split()
        if tup:
            fileO.write(line)
    fileI.close()
    fileO.close() 


##### L'objectif est mainteant de se débarasser de cette erreur

Warning: the reference and the candidate consists of different number of lines!

In [7]:
def adaptData(fileIn, fileOut):
    # compte le nombre de mot sur la ligne et recréer des lignes vides ensuite
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')
    
    for line in fileI:
        tup = line.split(" ")
        spaces = len(tup)
        fileO.write(line)
        for i in range (0, spaces-2):
            fileO.write("\n")
    fileI.close()
    fileO.close()

In [8]:
fileRefInter = "pos_reference.txt.univ.modif"
fileRef = "pos_reference.txt.univ"
fileRefOut = "pos_reference.txt.univ.adjustLines"

suppressEmptyLine(fileRef, fileRefInter)
adaptData(fileRefInter,fileRefOut)

##### Ligne de commande à taper pour réaliser les evaluate
-  python evaluate.py pos_test.txt.pos.stanford.univ pos_reference.txt.univ.adjustLines
-  python evaluate.py pos_test.txt.pos.nltk.univ pos_reference.txt.univ.modif

##### 6) Quelles conclusions vous pouvez avoir à partir des résultats d’évaluation des deux POS taggers. 

## <u> II. Evaluation de la reconnaissance d’entités nommées </u>

##### 1) Utiliser le corpus annoté « ne_reference.txt.conll » pour extraire les phrases ayant servi pour produire ce corpus annoté et sauvegarder le résultat dans le fichier « ne_test.txt ». Dans ce corpus, une ligne vide indique la fin de la phrase courante. 
 

In [1]:
def getTextPartieDeux(fileIn, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')

    for line in fileI:

        tuples = line.split()
        if tuples:
            fileO.write(tuples[0]+" ")
        else:
            # Si on arrive à une ligne vide on saute une ligne c'est un changement de phrase 
            fileO.write("\n")

    fileI.close()
    fileO.close()

In [6]:
fileIn = "ne_reference.txt.conll.txt"
fileOut = "ne_test.txt"
getTextPartieDeux(fileIn,fileOut)

##### 2) Lancer les deux NE recognizers sur le fichier « ne_test.txt ». Les résultats doivent avoir le format  du  corpus  annoté  « ne_reference.txt.conll »  (2  colonnes  séparées  par  une tabulation) et doivent être sauvegardés respectivement dans les fichiers suivants : 
-  ne_test.txt.ne.stanford 
-  ne_test.txt.ne.nltk 

In [20]:
import nltk
from nltk.tokenize import word_tokenize

def nltkTokenizeNE(fileIn,fileOut):
    
    file = open(fileIn, "r")
    fileO = open(fileOut, "w")
    for line in file:
        
        text = word_tokenize(line)
    
        tokens_tag = nltk.pos_tag(text)

        ne_tree = nltk.ne_chunk(tokens_tag, binary= False)
        
        for tree in ne_tree.subtrees() :
            
            if tree.label() == "PERSON":
                for branch in tree:
                    b1 = str(branch[0])
                    fileO.write(b1+" ")
                fileO.write("PERS")
                fileO.write("\n")
            elif tree.label() == "ORGANIZATION" or tree.label() == "FACILITY":
                for branch in tree:
                    b1 = str(branch[0])
                    fileO.write(b1+" ")
                fileO.write("ORG")
                fileO.write("\n")
            elif tree.label() == "LOCATION" or tree.label() == "GPE" or  tree.label() == "GSP" :
                for branch in tree:
                    b1 = str(branch[0])
                    fileO.write(b1+" ")
                fileO.write("LOC")
                fileO.write("\n")
            elif tree.label() == "MONEY" or tree.label() == "PERCENT" or tree.label() == "TIME" or  tree.label() == "DATE":
                for branch in tree:
                    b1 = str(branch[0])
                    fileO.write(b1+" ")
                fileO.write("MISC")
                fileO.write("\n")
    fileO.close()
    file.close()     

In [21]:
fileIn = "ne_test.txt"
fileOut = "ne_test.txt.ne.nltk"

nltkTokenizeNE(fileIn, fileOut)

On tape la commande suivante dans un terminal pour appliquer le NE recognizer de standford sur "ne_test.txt" :

java -mx600m -cp stanford-ner.jar:lib/* edu.stanford.nlp.ie.crf.CRFClassifier -loadClassifier classifiers/english.all.3class.distsim.crf.ser.gz -textFile  ../ne_test.txt > ../ne_test.txt.ne.stanford

In [26]:
# Faire une fonction pour que le le texte soit écrit en 2 colonnes
def miseEnPage(fileIn, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')
    for l in fileI:
        tup = l.split()
        for word in tup:
            for letter in word:
                if(letter == "/"):
                    fileO.write("\t")
                else : 
                    fileO.write(letter)
            fileO.write('\n')
    fileI.close()
    fileO.close()

In [27]:
fileIn = "ne_test.txt.ne.stanford"
fileOut = "ne_test.txt.ne.stanford.final"
miseEnPage(fileIn,fileOut)

Mary/PERSON
Barra/PERSON
appointed/O
as/O
General/ORGANIZATION
Motors/ORGANIZATION
chief/O
December/O
10/O
,/O
2013/O
The/O
United/LOCATION
States/LOCATION
's/O
largest/O
car/O
manufacturer/O
General/ORGANIZATION
Motors/ORGANIZATION
today/O
named/O
Mary/PERSON
Barra/PERSON
as/O
its/O
new/O
chief/O
executive/O
./O
Barra/PERSON
,/O
51/O
,/O
will/O
be/O
the/O
first/O
woman/O
to/O
lead/O
a/O
firm/O
in/O
the/O
American/O
auto/O
industry/O
./O
She/O
has/O
been/O
with/O
General/ORGANIZATION
Motors/ORGANIZATION
for/O
33/O
years/O
,/O
and/O
is/O
currently/O
the/O
global/O
head/O
of/O
product/O
development/O
./O
She/O
said/O
in/O
a/O
statement/O
:/O
``/O
With/O
an/O
amazing/O
portfolio/O
of/O
cars/O
and/O
trucks/O
and/O
the/O
strongest/O
financial/O
performance/O
in/O
our/O
recent/O
history/O
,/O
this/O
is/O
an/O
exciting/O
time/O
at/O
today/O
's/O
GM/O
./O
I/O
'/O
m/O
honored/O
to/O
lead/O
the/O
best/O
team/O
in/O
the/O
business/O
and/O
to/O
keep/O
our/O
momentum/O
at/O
full/O
speed/O
./O
''/O


##### 3) Convertir  les  résultats  des  deux  NE  recognizers  en  utilisant  les  étiquettes  CoNLL-2003 (https://www.clips.uantwerpen.be/conll2003/ner/  -Annexe  2-).  Les  résultats  de  cette conversion doivent être sauvegardés respectivement dans les fichiers suivants : 
-  ne_test.txt.ne.stanford.conll 
-  ne_test.txt.ne.nltk.conll 

##### 4) Lancer  l’évaluation  des  deux  NE  recognizers.  Pour  réaliser  cette  évaluation,  il  faut supprimer  la  ligne  vide  (séparant  les  phrases)  dans  le  de  fichier  de  référence « ne_reference.txt.conll »: 
-  python evaluate.py ne_test.txt.ne.stanford.conll ne_reference.txt.conll 
-  python evaluate.py ne_test.txt.ne.nltk.conll ne_reference.txt.conll 

##### 5) Quelles conclusions vous pouvez avoir à partir des résultats d’évaluation des deux NE recognizers.